# 一元线性回归
> from scipy.stats import linregress [摸鱼的相关分析和回归分析](https://www.heywhale.com/mw/project/61162916fe727700176cac60)

> 相关分析与回归分析都是统计上研究变量之间关系的常用方法。用来判断两组变量之间的统计相关性。相关分析中两组变量的地位是平等的，而回归分析两个变量位置一般不能互换(自变量和因变量)。

> linregress函数返回5个值，分别为回归斜率，回归截距，相关系数，p-value，估计斜率的标准误差。  
> slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from scipy.stats import linregress

In [2]:
def read_data_nc1(yr):
    global lat, lon
    inpath=("D:/study/Gosif_Data_0.5/phenology/EuroAsia_dbl_derive2rd_%d.nc"%yr)
    with nc.Dataset(inpath, mode='r') as f:
        #print(f.variables.keys())
        SOP=(f.variables['SOP'][:])
        lat=(f.variables['lat'][:])
        lon=(f.variables['lon'][:])
        #print(f.variables['sif'])
        SOP[SOP==-9999]=np.nan
    return SOP

In [4]:
#%%读取数据
for yr in range(2001, 2021):
        sop=read_data_nc1(yr)
        sop=sop.reshape(1, 120, 240)
        if yr == 2001:
            sop_all= sop
        else:
            sop_all= np.vstack((sop_all, sop))


#%%
t=np.arange(1, 21, 1)
yr=np.arange(2001, 2021, 1)
s,r,p = np.zeros((120, 240)),np.zeros((120, 240)),np.zeros((120, 240))

for i in range(len(lat)):
    for j in range(len(lon)):
        s[i,j],_,r[i,j], p[i,j],_  = linregress(t, sop_all[:,i,j])

## 标准误(Standard Error)
> [与R-square类似，衡量模型拟合程度](https://zhuanlan.zhihu.com/p/42074332)  

> 计算公式 SE(b) = sqrt(Σ(y - ŷ)^2 / (n - 2)) / sqrt(Σ(x - x̄)^2)  
> 其中，SE(b) 表示回归系数的标准误，y 是观测到的因变量值，ŷ 是根据回归模型预测的因变量值，n 是样本大小，x 是自变量值，x̄ 是自变量的均值。  

> 例如，假设我们的回归分析结果显示房屋面积的回归系数为 50，标准误为 5。这意味着我们估计的回归系数为每增加 1 平方米的房屋面积，房屋价格平均增加 50 美元。而标准误为 5 表示我们的估计值相对准确，有较高的置信度。  
置信区间是用来估计回归系数真实值的范围。通常使用回归系数的估计值加减两倍标准误来计算置信区间。在这种情况下，回归系数的置信区间为 50 ± (2 * 5)，即 (40, 60)。

In [1]:
import numpy as np
import statsmodels.api as sm

# 示例数据
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 5, 6, 8])

# 添加常数列，构建回归模型
X = sm.add_constant(x)

# 执行一元线性回归
model = sm.OLS(y, X)
results = model.fit()

# 输出回归系数的标准误
print("标准误:", results.bse[1])

标准误: 0.11547005383792515


## 临界回归系数
1. 确定显著性水平（通常以 α 表示），例如 0.05 表示 5% 的显著性水平。
2. 确定自由度（通常是样本大小减去回归模型中的参数个数）。
3. 使用 t 分布的累积分布函数（CDF）或查找 t 分布表，找到对应显著性水平和自由度的 t 分布临界值。
4. 根据回归模型的标准误，计算临界回归系数。

In [2]:
from scipy import stats

# 示例数据
alpha = 0.05  # 显著性水平
n = 50  # 样本大小
df = n - 2  # 自由度
se = 0.5  # 回归系数的标准误

# 计算临界回归系数
critical_value = stats.t.ppf(1 - alpha / 2, df) * se

print("临界回归系数:", critical_value)

临界回归系数: 1.0053173773482227


# 多元线性回归